# 3. Hill-Tononi model of sleep

[Notebook source](https://github.com/synergetics/nestlings/blob/master/4_hill_tononi.ipynb)

This tutorial shows you how to implement a simplified version of the Hill-Tononi model of the early visual pathway using the NEST Topology module. The model is described in the paper

S. L. Hill and G. Tononi. Modeling Sleep and Wakefulness in the Thalamocortical System. J Neurophysiology 93:1671-1698 (2005). Freely available via `doi 10.1152/jn.00915.2004 http://dx.doi.org/10.1152/jn.00915.2004

**Model simplifications**

We simplify the model somewhat both to keep this tutorial a bit shorter, and because some details of the Hill-Tononi model are not currently supported by NEST. Simplifications include:

1. We use the iaf_cond_alpha neuron model, which is simpler than the Hill-Tononi model.

2. As the iaf_cond_alpha neuron model only supports two synapses (labeled "ex" and "in"), we only include AMPA and GABA_A synapses.

3. We ignore the secondary pathway (Ts, Rs, Vs), since it adds just more of the same from a technical point of view.

4. Synaptic delays follow a Gaussian distribution in the HT model. This implies actually a Gaussian distributions clipped at some small, non-zero delay, since delays must be positive. Currently, there is a bug in the Topology module when using clipped Gaussian distribution. We therefore draw delays from a uniform distribution.

5. Some further adaptations are given at the appropriate locations in the script.

## Philosophy

A network models has two essential components: populations and projections. We first use NEST's CopyModel() mechanism to create specific models for all populations and subpopulations in the network, and then create the populations using the Topology modules CreateLayer() function.

We use a two-stage process to create the connections, mainly because the same configurations are required for a number of projections: we first define dictionaries specifying the connections, then apply these dictionaries later.

## Preparations

In [1]:
SHOW_FIGURES = True

import pylab

if not SHOW_FIGURES:
    pylab_show = pylab.show

    def nop(s=None):
        pass

    pylab.show = nop
else:
    pylab.ion()

In [2]:
import math
import nest

nest.ResetKernel()

<frozen importlib._bootstrap>:241: RuntimeWarning: compiletime version 3.8 of module 'pynestkernel' does not match runtime version 3.1
<frozen importlib._bootstrap>:241: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 880 from C header, got 888 from PyObject



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.3
 Built: May  5 2022 07:35:57

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



## Configurable Parameters

The parameters represent
* Network size in neurons N, each layer is N x N.
* Network size in subtended visual angle visSize, in degree.
* Temporal frequency of drifting grating input f_dg, in Hz.
* Spatial wavelength and direction of drifting grating input, lambda_dg and phi_dg, in degree/radian.
* Background firing rate of retinal nodes and modulation amplitude, retDC and retAC, in Hz.
* Simulation duration simtime; actual simulation is split into intervals of sim_interval length, so that the network state can be visualized in those intervals. Times are in ms.

In [3]:
params = {
    "N": 40,
    "visSize": 8.0,
    "f_dg": 2.0,
    "lambda_dg": 2.0,
    "phi_dg": 0.0,
    "retDC": 30.0,
    "retAC": 30.0,
    "simtime": 100.0,
    "sim_interval": 5.0,
}

## Neuron Models

Some simplifications are done here w.r.t to the original paper. See the notebook source.

In [4]:
# BASE NERUON MODEL
nest.CopyModel(
    "iaf_cond_alpha",
    "NeuronModel",
    params={
        "C_m": 16.0,
        "E_L": (0.2 * 30.0 + 1.5 * -90.0) / (0.2 + 1.5),
        "g_L": 0.2 + 1.5,
        "E_ex": 0.0,
        "E_in": -70.0,
        "V_reset": -60.0,
        "V_th": -51.0,
        "t_ref": 2.0,
        "tau_syn_ex": 1.0,
        "tau_syn_in": 2.0,
        "I_e": 0.0,
        "V_m": -70.0,
    },
)

# Cortical excitatory cells
nest.CopyModel("NeuronModel", "CtxExNeuron")

# Cortical inhibitory cells
nest.CopyModel(
    "NeuronModel", "CtxInNeuron", params={"C_m": 8.0, "V_th": -53.0, "t_ref": 1.0}
)

# Thalamic cells
nest.CopyModel(
    "NeuronModel",
    "ThalamicNeuron",
    params={"C_m": 8.0, "V_th": -53.0, "t_ref": 1.0, "E_in": -80.0},
)

## Input generating nodes

Input is generated by sinusoidally modulating Poisson generators, organized in a square layer of retina nodes. These nodes require a slightly more complicated initialization than all other elements of the network:

* Average firing rate DC, firing rate modulation depth AC, and temporal modulation frequency Freq are the same for all retinal nodes and are set directly below.
* The temporal phase Phi of each node depends on its position in the grating and can only be assigned after the retinal layer has been created. We therefore specify a function for initalizing the phase Phi. This function will be called for each node.

In [5]:
def phiInit(pos, lam, alpha):
    """Initializer function for phase of drifting grating nodes.

    pos  : position (x,y) of node, in degree
    lam  : wavelength of grating, in degree
    alpha: angle of grating in radian, zero is horizontal

    Returns number to be used as phase of AC Poisson generator.
    """
    return 2.0 * math.pi / lam * (math.cos(alpha) * pos[0] + math.sin(alpha) * pos[1])


nest.CopyModel(
    "sinusoidal_poisson_generator",
    "RetinaNode",
    params={
        "ac": params["retAC"],
        "dc": params["retDC"],
        "freq": params["f_dg"],
        "phi": 0.0,
        "individual_spike_trains": False,
    },
)

## Recording nodes

In [6]:
nest.CopyModel(
    "multimeter",
    "RecordingNode262",
    {
        "interval": params["sim_interval"],
        "record_from": ["V_m"],
        "record_to": "memory",
    },
)

## Populations

In [8]:
layerProps = {
    "rows": params["N"],
    "columns": params["N"],
    "extent": [params["visSize"], params["visSize"]],
    "edge_wrap": True,
}
# This dictionary does not yet specify the elements to put into the
# layer, since they will differ from layer to layer. We will add them
# below by updating the `'elements'` dictionary entry for each
# population.

**Retina**

In [ ]:
layerProps.update({"elements": "RetinaNode"})
retina = topo.CreateLayer(layerProps)

# Now set phases of retinal oscillators; we use a list comprehension instead
# of a loop.
[
    nest.SetStatus(
        [n],
        {
            "phi": phiInit(
                topo.GetPosition([n])[0], params["lambda_dg"], params["phi_dg"]
            )
        },
    )
    for n in nest.GetLeaves(retina)[0]
]

In [ ]:
nest.GetLeaves(retirna)